In [1]:
import numpy as np
import pandas as pd
import re

pd.options.display.max_columns = None

In [2]:
def find_headers(dataframe):
    # Search Date/time header
    header_idx = -1
    for ri, rows in enumerate(dataframe.values):
        for ri, row in enumerate(rows):
            is_header = 'Date/Time'.lower() in str(row).lower()
            if is_header:
                header_idx = ri + 1
                break

    # Check if a valid header index was found
    if header_idx != -1:
        print("Header found at row index:", header_idx)
        # Set the header and skip rows accordingly
        dataframe = pd.DataFrame(dataframe.values[header_idx:], columns=dataframe.iloc[header_idx - 1])
        dataframe = dataframe.reset_index(drop=True)
    else:
        print("Header not found in the DataFrame")
    return dataframe


def find_footer(dataframe):
    row_index = None
    for i, row in dataframe.iterrows():
        if 'Station Down (D)=' in str(row['Date/Time']):
            row_index = i
            break

    print('Remove footer at row index:', row_index)

    # Remove rows from the bottom up to and including the found row
    if row_index is not None:
        print("Removing rows from index:", row_index, -1)
        dataframe = dataframe.iloc[:row_index]

        # CLean NaN last 10 rows
        df_no10 = dataframe.iloc[:-10]
        df_cleaned = dataframe.tail(10).dropna(subset=['Date/Time'], how='any')
        dataframe = pd.concat([df_no10, df_cleaned], ignore_index=True)

    else:
        print("Text 'Station Down (D)=' not found")

    # Reset the index if needed
    dataframe = dataframe.reset_index(drop=True)
    return dataframe

In [3]:
uri = r'D:\Work\Home-2023\may-project\project-mei\ข้อมูลย้อนหลัง10ปี\ผลการตรวจวัดคุณภาพอากาศ\PM10 2009-2019\Banhuafai.xlsx'
uri = r'D:\Work\Home-2023\may-project\project-mei\ข้อมูลย้อนหลัง10ปี\ผลการตรวจวัดคุณภาพอากาศ\NO2 2009-2019\Banhuafai.xlsx'
uri = r'D:\Work\Home-2023\may-project\project-mei\ข้อมูลย้อนหลัง10ปี\ผลการตรวจวัดคุณภาพอากาศ\NO2 2009-2019\BanHuayking.xlsx'
# uri = r'D:\Work\Home-2023\may-project\project-mei\ข้อมูลย้อนหลัง10ปี\ผลการตรวจวัดคุณภาพอากาศ\PM10 2009-2019\Banhuafai.xlsx'
xls = pd.ExcelFile(uri)

print('Sheet nums', xls.sheet_names)
dfs = []

for sheet_name in xls.sheet_names:
    # Read data from the current sheet
    df = pd.read_excel(xls, sheet_name=sheet_name,)
    df = find_headers(df)
    df = find_footer(df)
    dfs.append(df)

concatenated_df = pd.concat(dfs, ignore_index=True)
df = concatenated_df

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Work\\Home-2023\\may-project\\project-mei\\ข้อมูลย้อนหลัง10ปี\\ผลการตรวจวัดคุณภาพอากาศ\\NO2 2009-2019\\BanHuayking.xlsx'

In [ ]:
df.info()

In [ ]:
row_index = None
for i, row in df.iterrows():
    if 'Station Down (D)=' in str(row['Date/Time']):
        row_index = i
        break

print('Remove footer at row index:', row_index)

# Remove rows from the bottom up to and including the found row
if row_index is not None:
    print("Removing rows from index:", row_index, -1)
    df = df.iloc[:row_index - 1]
else:
    print("Text 'Station Down (D)=' not found")

# Reset the index if needed
df = df.reset_index(drop=True)

In [ ]:
str_fill = ['F', 'C', 'D', 'A', 'P', '-', 'O']
df = df.replace(str_fill, [np.NaN] * str_fill.__len__())
# df = df.ffill().bfill()

In [ ]:
df.columns

In [ ]:
# Use a regular expression to filter columns with the pattern 'xx:xx'
time_pattern = r'\d{2}:\d{2}(:\d{2})*'
time_columns = [col for col in df.columns if re.search(time_pattern, str(col))]
if len(time_columns) == 24:  # Check if you have 24 time columns
    start_time = '1990-01-01 01:00:00'
    end_time = '1990-01-02 00:59:59'
    time_range = pd.date_range(start=start_time, end=end_time, freq='1H')
    time_columns = [time.strftime('%H:%M:%S') for time in time_range.time]
else:
    raise ValueError('The number of time columns is not 24')

df = df.iloc[:,:25]
df.columns = ['Date/Time'] + time_columns

df

In [ ]:
# Use melt to flatten the selected columns into rows
melted_df = pd.melt(df, id_vars=['Date/Time'], var_name='Time', value_name='Value')
melted_df



In [ ]:
melted_df['Datetime'] = pd.to_datetime(melted_df['Date/Time'].astype(str), format='%Y-%m-%d', errors='coerce')
melted_df['_time_dt'] = pd.to_datetime(melted_df['Time'], format='%H:%M:%S', errors='coerce').dt.time
melted_df['Datetime'] += pd.to_timedelta(melted_df['_time_dt'].astype(str))
melted_df.loc[melted_df['Time'] == '00:00:00', 'Datetime'] += pd.DateOffset(days=1)



# melted_df = melted_df[['Datetime', 'Value']]
melted_df

In [ ]:
df_sorted = melted_df.sort_values(by='Datetime')
df_sorted.reset_index(drop=True, inplace=True)
melted_df = df_sorted

# Extract years from the Datetime column
melted_df['Year'] = melted_df['Datetime'].dt.year
unique_years = melted_df['Year'].unique()

melted_df

In [ ]:
melted_df['ValueLn'] = melted_df['Value'].interpolate(method='linear')
melted_df

In [ ]:
missing_values = melted_df.isna().sum()
summary_stats = melted_df.describe()
unique_values = melted_df['Datetime'].unique()
value_counts = melted_df['Datetime'].value_counts()
yearly_sum = melted_df.groupby('Year').count()
missing_values

In [ ]:
summary_stats

In [ ]:
yearly_sum

In [ ]:
melted_df[melted_df['Value'].isna()]


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(melted_df['Datetime'], melted_df['Value'], linestyle='-', lw=0.5)
plt.xlabel('Datetime')
plt.ylabel('Value')
plt.title('Time Series Plot')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:

# Plot seasonality for each year
plt.figure(figsize=(360, 12))
# First subplot on the left

for year in unique_years:
    year_data = melted_df[melted_df['Year'] == year]
    plt.subplot(2, 1, 1)
    plt.plot(year_data['Datetime'], year_data['ValueLn'], label=f'Year {year}', lw=0.5)
    plt.ylabel('Value')
    plt.subplot(2, 1, 2)
    plt.plot(year_data['Datetime'], year_data['Value'], label=f'Year {year}', lw=0.5)
    plt.ylabel('Value')

plt.xlabel('Datetime')

plt.title('Seasonality Plot for Each Year')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend(loc='upper right')

plt.tight_layout()

# Show the plot
plt.savefig('Seasonality.png')